In [1]:
import mlflow
import requests
import pandas as pd
import mlflow.pyfunc
from sklearn.preprocessing import StandardScaler
import joblib

In [2]:
df = pd.read_csv('/home/project/data/simulados/simulados.csv')
df = df.drop(['TIPO_PACIENTE',], axis=1)
scaler = joblib.load('/home/project/models/preprocess_utils/scaler.pkl')
encoder = joblib.load('/home/project/models/preprocess_utils/encoder.pkl')
encoded_columns  = encoder.transform(df[['SEXO', 'ESTATUS_CASO', 'RESULTADO_PCR']])
scaled_columns = scaler.fit_transform(df.drop(['SEXO', 'ESTATUS_CASO', 'RESULTADO_PCR'], axis=1))
encoded_scaled_df = pd.DataFrame(encoded_columns, columns=encoder.get_feature_names_out(['SEXO', 'ESTATUS_CASO', 'RESULTADO_PCR']))
scaled_df = pd.DataFrame(scaled_columns, columns=df.drop(['SEXO', 'ESTATUS_CASO', 'RESULTADO_PCR'], axis=1).columns)
data = pd.concat([encoded_scaled_df, scaled_df], axis=1)

El modelo se hara accesible a traves de una API creada en ModelAPI.py con la libreria Fast API

In [4]:
! python /home/project/deploy/ModelAPI.py

INFO:     Started server process [46404]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
^C
INFO:     Shutting down


Se hace el request al puerto donde esta el modelo.

In [ ]:
# URL del servicio local
url = "http://localhost:8000/predict/"

# Enviar una solicitud POST al servidor
response = requests.post(url, json={"data": data.tolist()})

# Imprimir la respuesta
print(response.json())



{'prediction': [1, 1, 1, 1, 1]}
